# K-Fold Cross Validation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook as tqdm

## Data

In [2]:
breast_cancer = pd.read_csv('../Data/breast_cancer.csv')
breast_cancer.drop(['id'], axis=1, inplace=True)
breast_cancer_dummies = pd.get_dummies(breast_cancer, columns=['diagnosis'], drop_first=True)
breast_cancer_dummies.rename(columns={'diagnosis_M': 'Malignant'}, inplace=True)
breast_cancer_dummies.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Malignant
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,1
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,1
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,1
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,1


In [3]:
inputs = breast_cancer_dummies.drop(['Malignant'],axis=1)
target = breast_cancer_dummies.Malignant

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs, target, test_size=0.2)

# Model Import

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

## Logistic Regression

In [6]:
logRegression = LogisticRegression(max_iter=5000)
logRegression.fit(X_train, y_train)
logRegression.score(X_test, y_test)

0.9210526315789473

## Support Vector Machine

In [7]:
svm = SVC()
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

0.8771929824561403

## Random Forrest

In [14]:
randomForrest = RandomForestClassifier()
randomForrest.fit(X_train, y_train)
randomForrest.score(X_test, y_test)

0.9157894736842105

## X_train, X_test random samples from database
X_Train, X_test etc will represent random samples from the database, thus each time we retrieve new samples our models performance/score will
change. Thus, we cannot evaluate the models performance solely based on one random subset, we have to run the model multiple times on different subsets of data

## K-Fold API

In [15]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)
kf

KFold(n_splits=3, random_state=None, shuffle=False)

In [16]:
for train_index, test_index in kf.split([0,1,2,3,4,5,6,7,8,9]):
    print(train_index, test_index)

[4 5 6 7 8 9] [0 1 2 3]
[0 1 2 3 7 8 9] [4 5 6]
[0 1 2 3 4 5 6] [7 8 9]


In [17]:
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

## Stratified K-Fold
We will StratifiedKFold instead of KFold as it allows the classification classes e.g Malignant and Benign to be equally distributed in our samples

In [18]:
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=3)

In [19]:
from sklearn.preprocessing import StandardScaler
inputs_transformed =StandardScaler().fit_transform(inputs)
inputs_transformed

array([[ 1.09706398, -2.07333501,  1.26993369, ...,  2.29607613,
         2.75062224,  1.93701461],
       [ 1.82982061, -0.35363241,  1.68595471, ...,  1.0870843 ,
        -0.24388967,  0.28118999],
       [ 1.57988811,  0.45618695,  1.56650313, ...,  1.95500035,
         1.152255  ,  0.20139121],
       ...,
       [ 0.70228425,  2.0455738 ,  0.67267578, ...,  0.41406869,
        -1.10454895, -0.31840916],
       [ 1.83834103,  2.33645719,  1.98252415, ...,  2.28998549,
         1.91908301,  2.21963528],
       [-1.80840125,  1.22179204, -1.81438851, ..., -1.74506282,
        -0.04813821, -0.75120669]])

In [20]:
scoresLogRegression = []
scoresSVM = []
scoreRandomForrest = []

for train_index, test_index in  kf.split(inputs_transformed):
    # The loop will repeat 3 times as n_splits=3
    X_train, X_test, y_train, y_test = inputs_transformed[train_index], inputs_transformed[test_index], \
                                        target[train_index], target[test_index]
    # Append score to above model array
    scoresLogRegression.append(get_score(LogisticRegression(), X_train, X_test, y_train, y_test))
    scoresSVM.append(get_score(SVC(), X_train, X_test, y_train, y_test))
    scoreRandomForrest.append(get_score(RandomForestClassifier(n_estimators=100), X_train, X_test, y_train, y_test))

print("Logistic Regression: ", np.mean(scoresLogRegression))
print("SVM: ", np.mean(scoresSVM))
print("Random Forrest: ", np.mean(scoreRandomForrest))

Logistic Regression:  0.9718834122342894
SVM:  0.9666295368049754
Random Forrest:  0.9613570964448157


## Cross Validation Score SkLearn
from sklearn.model_selection import cross_val_score

In [21]:
np.mean(cross_val_score(LogisticRegression(), inputs_transformed, target, cv=10))

NameError: name 'cross_val_score' is not defined

In [22]:
np.mean(cross_val_score(SVC(), inputs_transformed, target, cv=10))

NameError: name 'cross_val_score' is not defined

In [23]:
np.mean(cross_val_score(RandomForestClassifier(n_estimators=50), inputs_transformed, target, cv=10))

NameError: name 'cross_val_score' is not defined